<a href="https://colab.research.google.com/github/amirimmd/Machine_Learnin/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1**

---



In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, layer_sizes, activation_function='sigmoid', learning_rate=0.01):
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes)
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i-1]) for i in range(1, self.num_layers)]
        self.biases = [np.zeros((layer_sizes[i], 1)) for i in range(1, self.num_layers)]
        self.activation_function = activation_function
        self.learning_rate = learning_rate

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def _sigmoid_derivative(self, z):
        return self._sigmoid(z) * (1 - self._sigmoid(z))

    def _relu(self, z):
        return np.maximum(0, z)

    def _relu_derivative(self, z):
        return np.where(z > 0, 1, 0)

    def _forward_propagation(self, X):
        activations = [X]
        weighted_sums = []

        for i in range(self.num_layers - 1):
            weighted_sum = np.dot(self.weights[i], activations[i]) + self.biases[i]
            weighted_sums.append(weighted_sum)

            if self.activation_function == 'sigmoid':
                activations.append(self._sigmoid(weighted_sum))
            elif self.activation_function == 'relu':
                activations.append(self._relu(weighted_sum))

        return activations, weighted_sums

    def _backward_propagation(self, X, y, activations, weighted_sums):
        m = X.shape[1]
        delta = (activations[-1] - y) * self._sigmoid_derivative(weighted_sums[-1])

        biases_gradients = [np.sum(delta, axis=1, keepdims=True) / m]
        weights_gradients = [np.dot(delta, activations[-2].T) / m]

        for i in range(2, self.num_layers):
            delta = np.dot(self.weights[-i+1].T, delta) * \
                    (self._sigmoid_derivative(weighted_sums[-i]) if self.activation_function == 'sigmoid' else self._relu_derivative(weighted_sums[-i]))

            biases_gradients.insert(0, np.sum(delta, axis=1, keepdims=True) / m)
            weights_gradients.insert(0, np.dot(delta, activations[-i-1].T) / m)

        return biases_gradients, weights_gradients

    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            activations, weighted_sums = self._forward_propagation(X)

            biases_gradients, weights_gradients = self._backward_propagation(X, y, activations, weighted_sums)

            for i in range(self.num_layers - 1):
                self.weights[i] -= self.learning_rate * weights_gradients[i]
                self.biases[i] -= self.learning_rate * biases_gradients[i]

    def predict(self, X):
        activations, _ = self._forward_propagation(X)
        return activations[-1]

# Determining the network architecture (number of nodes in each layer)
layer_sizes = [2, 4, 1]

# Building a network with a specified number of layers, activity function and learning rate
nn = NeuralNetwork(layer_sizes, activation_function='sigmoid', learning_rate=0.1)

# Generating synthetic input and output data
X_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
y_train = np.array([[0, 1, 1, 0]])

# Network training with input and output data
nn.train(X_train, y_train, epochs=10000)

# Output prediction for new inputs
X_new = np.array([[0, 1], [1, 1]]).T
predictions = nn.predict(X_new)

print("Predictions:", predictions)


Predictions: [[0.86004251 0.17083747]]


# **2**

---



In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, activation_function='sigmoid', learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.weights_input_hidden = np.random.randn(hidden_size, input_size)
        self.biases_hidden = np.zeros((hidden_size, 1))
        self.weights_hidden_output = np.random.randn(output_size, hidden_size)
        self.biases_output = np.zeros((output_size, 1))
        self.activation_function = activation_function
        self.learning_rate = learning_rate

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def _sigmoid_derivative(self, z):
        return self._sigmoid(z) * (1 - self._sigmoid(z))

    def _relu(self, z):
        return np.maximum(0, z)

    def _relu_derivative(self, z):
        return np.where(z > 0, 1, 0)

    def _forward_propagation(self, X):
        self.hidden_layer_input = np.dot(self.weights_input_hidden, X) + self.biases_hidden
        self.hidden_layer_output = self._sigmoid(self.hidden_layer_input) if self.activation_function == 'sigmoid' else self._relu(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.weights_hidden_output, self.hidden_layer_output) + self.biases_output
        self.output_layer_output = self._sigmoid(self.output_layer_input)

        return self.output_layer_output

    def _backward_propagation(self, X, y):
        m = X.shape[1]

        # Calculate output layer gradients
        output_layer_error = self.output_layer_output - y
        output_layer_delta = output_layer_error * self._sigmoid_derivative(self.output_layer_input)

        # Update output layer weights and biases
        self.weights_hidden_output -= self.learning_rate * np.dot(output_layer_delta, self.hidden_layer_output.T) / m
        self.biases_output -= self.learning_rate * np.sum(output_layer_delta, axis=1, keepdims=True) / m

        # Calculate hidden layer gradients
        hidden_layer_error = np.dot(self.weights_hidden_output.T, output_layer_delta)
        hidden_layer_delta = hidden_layer_error * (self._sigmoid_derivative(self.hidden_layer_input) if self.activation_function == 'sigmoid' else self._relu_derivative(self.hidden_layer_input))

        # Update hidden layer weights and biases
        self.weights_input_hidden -= self.learning_rate * np.dot(hidden_layer_delta, X.T) / m
        self.biases_hidden -= self.learning_rate * np.sum(hidden_layer_delta, axis=1, keepdims=True) / m

    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            for i in range(X.shape[1]):
                input_data = X[:, i].reshape(-1, 1)
                target_output = y[:, i].reshape(-1, 1)

                self._forward_propagation(input_data)
                self._backward_propagation(input_data, target_output)

    def predict(self, X):
        predictions = []
        for i in range(X.shape[1]):
            input_data = X[:, i].reshape(-1, 1)
            output = self._forward_propagation(input_data)
            predictions.append(output)

        return np.hstack(predictions)

# Download MNIST data
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Resize data to (number of samples, image dimensions)
X_train = X_train.reshape((X_train.shape[0], -1)).T / 255.0
X_test = X_test.reshape((X_test.shape[0], -1)).T / 255.0

# Convert tags to one-hot encoding
y_train_one_hot = np.eye(10)[y_train].T
y_test_one_hot = np.eye(10)[y_test].T

# Determining the network architecture (number of nodes in each layer)
input_size = X_train.shape[0]
hidden_size = 64
output_size = 10

# Building the network with the number of nodes in each layer, the activity function and the specified learning rate
nn = NeuralNetwork(input_size, hidden_size, output_size, activation_function='sigmoid', learning_rate=0.1)

# Network training with training data
nn.train(X_train, y_train_one_hot, epochs=5)

# Output prediction for test data
predictions = nn.predict(X_test)

# Calculate accuracy
accuracy = np.sum(np.argmax(predictions, axis=0) == y_test) / y_test.shape[0]
print("Test Accuracy:", accuracy)

11490434/11490434 [==============================] - 0s 0us/step
Test Accuracy: 0.9334
